In [1]:
!pip install langchain langchain-community pypdf chromadb -q
!pip install langchain_groq -q
!pip install -U langchain-huggingface -q
!pip install -U langchain-chroma -q
!pip install gradio -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-groq 0.3.8 requires langchain-core<1.0.0,>=0.3.75, but you have langchain-core 1.0.7 which is incompatible.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.7 which is incompatible.


In [2]:
!pip install langchain langchain-core langchain-community langchain-groq chromadb sentence-transformers



  Using cached langchain_core-0.3.80-py3-none-any.whl.metadata (3.2 kB)
Using cached langchain_core-0.3.80-py3-none-any.whl (450 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.0.7
    Uninstalling langchain-core-1.0.7:
      Successfully uninstalled langchain-core-1.0.7
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-chroma 1.0.0 requires langchain-core<2.0.0,>=1.0.0, but you have langchain-core 0.3.80 which is incompatible.
langchain-huggingface 1.0.1 requires langchain-core<2.0.0,>=1.0.3, but you have langchain-core 0.3.80 which is incompatible.


In [3]:
# Importations
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

import google.generativeai as genai
from groq import Client  # Assurez-vous d'utiliser le bon SDK Groq

import os
import gradio as gr
# Définition du chemin d

In [4]:
# Définition du chemin du fichier PDF
pdf_path = "/content/Journal0132025.pdf"

# Chargement du document PDF
loader = PyPDFLoader(pdf_path)

# Extraction des pages
pages = loader.load()

# Vérification du nombre de pages chargées
print(f"Nombre de pages chargées : {len(pages)}")

# Affichage du contenu de la première page pour contrôle
print(pages[0].page_content[:500])  # Affiche les 500 premiers caractères de la première page

Nombre de pages chargées : 17
TRADUCTION FRANÇAISE POUR INFORMATION 
 
 
 
 
 
 
 
 
Sommaire 
 Décrets et arrêtés 
 
Présidence du Gouvernement 
Arrêté du Chef du Gouvernement du 30 janvier 2025, portant organisation et 
ouverture d'une session de formation au profit des hauts cadres 
administratifs à l'Institut de leadership administratif à l'Ecole nationale 
d'administration pour l'année 2025 ...................................................................  
Nomination de contrôleurs généraux des dépenses publiques ...


In [5]:
# Définition du découpeur de texte
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # Taille de chaque chunk
    chunk_overlap=50,  # Chevauchement entre les chunks pour conserver le contexte
    separators=["\n\n", "\n", " ", ""],  # Séparateurs pour découper propremen
)

# Découpage des pages en chunks
docs = text_splitter.split_documents(pages)

# Vérification
print(f"Nombre total de chunks créés : {len(docs)}")
print("\nExemple d'un chunk :\n")
print(docs[0].page_content)  # Afficher un exemple de chunk

Nombre total de chunks créés : 110

Exemple d'un chunk :

TRADUCTION FRANÇAISE POUR INFORMATION 
 
 
 
 
 
 
 
 
Sommaire 
 Décrets et arrêtés 
 
Présidence du Gouvernement 
Arrêté du Chef du Gouvernement du 30 janvier 2025, portant organisation et 
ouverture d'une session de formation au profit des hauts cadres 
administratifs à l'Institut de leadership administratif à l'Ecole nationale 
d'administration pour l'année 2025 ...................................................................


In [ ]:
import os
#os.environ["OPENAI_API_KEY"] = "*********************"

In [7]:
# Chemin où sauvegarder la base de données
persist_directory = "/kaggle/working/chroma_db"

# Chargement du modèle d'embeddings open-source
embedding_function = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Vérifier si une base Chroma existe déjà
if os.path.exists(persist_directory):
    print("Chargement de la base Chroma existante...")
    vectorstore = Chroma(persist_directory=persist_directory, embedding_function=embedding_function)
else:
    print("Création d'une nouvelle base Chroma...")
    vectorstore = Chroma.from_documents(docs, embedding_function, persist_directory=persist_directory)
    #vectorstore.persist()  # Sauvegarde de la base

# Vérification
print(f"Nombre de documents indexés : {vectorstore._collection.count()}")

/tmp/ipython-input-1062372533.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public mo

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Création d'une nouvelle base Chroma...
Nombre de documents indexés : 110


In [ ]:
#os.environ["GROQ_API_KEY"] = "*****"  # C'est ma clé, mettez la votre à la place
genai.configure(api_key=os.environ["GROQ_API_KEY"])

api_key = os.getenv("GROQ_API_KEY")

# Initialisation du client Groq avec la clé API depuis l'environnement
api_key = os.getenv("GROQ_API_KEY")
if not api_key:
    raise ValueError("La clé API GROQ_API_KEY n'est pas définie.")

client = Client(api_key=api_key)

In [13]:
# Initialisation du modèle Groq
groq_llm = ChatGroq(model="llama-3.3-70b-versatile")

# Création du prompt pour Groq basé sur le JORT Tunisien
prompt_template = (
    "Vous êtes un expert en droit tunisien et vous devez répondre avec précision "
    "en vous basant exclusivement sur le Journal Officiel de la République Tunisienne (JORT). "
    "Voici le contexte juridique extrait du JORT : {context} \n\n"
    "Question : {question} \n\n"
    "Fournissez une réponse claire, précise et conforme aux textes législatifs et réglementaires tunisiens."
)


# Définition de la chaîne QA avec le retriever
qa_chain = RetrievalQA.from_chain_type(
    llm=groq_llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    return_source_documents=True
)

In [14]:
# Exemple de question
question = "Pouvez-vous résumer les dernières réformes législatives publiées dans le JORT ?"


# Utilisation de `run()` pour obtenir la réponse et les documents sources
response = qa_chain.invoke({"query": question})


# Extraction des résultats
answer = response['result']
source_documents = response['source_documents']

# Affichage de la réponse et des documents sources
print(f"Réponse : {answer}")
print("Documents sources :")
for doc in source_documents:
  print(doc.page_content)  # Assurez-vous que vos documents ont l'attribut `page_content`

Réponse : Je ne peux pas résumer les dernières réformes législatives publiées dans le Journal Officiel de la République Tunisienne (JORT) car les informations fournies sont un extrait de textes législatifs spécifiques, mais je ne dispose pas d'informations en temps réel ou d'accès à la base de données du JORT.

Cependant, je peux vous indiquer que les textes fournis font référence à différentes lois et décrets, notamment :

* La loi n° 64-44 du 3 novembre 1964, portant réforme de l'école nationale d'administration
* La loi n° 83-112 du 12 décembre 1983, portant statut général des personnels de l'Etat
* La loi n° 2003-69 du 20 octobre 2003, relative aux centres de vacances d’été et de loisirs pour enfants
* Le décret n° 2004-2579 du 2 novembre 2004, portant création du centre 7 novembre de vacances d’été et de loisirs pour enfants à Hammamet
* L'arrêté du Chef du Gouvernement du 30 janvier 2025, portant organisation et ouverture d'une session de formation au profit des hauts cadres admi

In [15]:
sample_questions = [
    # Questions sur les lois et réglementations générales
    "Quelle est la dernière loi publiée dans le JORT sur le droit du travail ?",
    "Quelles sont les nouvelles mesures fiscales introduites cette année selon le JORT ?",
    "Quels sont les textes de loi régissant les marchés publics en Tunisie ?",
    "Existe-t-il des modifications récentes du Code des sociétés commerciales ?",
    "Quelle est la réglementation actuelle sur les importations en Tunisie selon le JORT ?",

    # Questions sur le droit du travail et social
    "Quelles sont les nouvelles dispositions du Code du travail publiées dans le JORT ?",
    "Quelle est la durée légale du travail en Tunisie selon les derniers textes officiels ?",
    "Quels sont les droits des employés en cas de licenciement selon le JORT ?",
    "Y a-t-il eu des changements récents concernant le salaire minimum en Tunisie ?",
    "Quelles sont les nouvelles mesures de protection sociale adoptées récemment ?",

    # Questions sur les marchés publics et entreprises
    "Quels sont les critères d’éligibilité pour répondre à un appel d’offres public en Tunisie ?",
    "Quelle est la procédure pour enregistrer une nouvelle entreprise en Tunisie selon le JORT ?",
    "Quels sont les avantages fiscaux pour les startups en Tunisie ?",
    "Quelles sont les obligations légales des entreprises en matière de déclaration fiscale ?",

    # Questions sur le droit immobilier et urbanisme
    "Quelles sont les nouvelles lois régissant l’acquisition de biens immobiliers en Tunisie ?",
    "Quels sont les critères pour bénéficier d’un lotissement social selon le JORT ?",
    "Quelles sont les autorisations nécessaires pour construire un immeuble en Tunisie ?",
    "Y a-t-il des modifications récentes dans la loi sur le bail commercial ?",

    # Questions sur les taxes et impôts
    "Quels sont les nouveaux taux d’imposition appliqués aux entreprises en Tunisie ?",
    "Quels sont les délais de déclaration fiscale pour les particuliers et entreprises ?",
    "Quelles exonérations fiscales sont prévues pour les investisseurs étrangers ?",
    "Y a-t-il eu des modifications dans la TVA selon les derniers décrets du JORT ?",

    # Questions sur les droits et obligations des citoyens
    "Quels sont les droits du consommateur selon les dernières lois publiées ?",
    "Comment obtenir une carte de séjour en Tunisie selon le JORT ?",
    "Quelle est la réglementation concernant importation de véhicules en Tunisie ?",
    "Quelles sont les nouvelles règles concernant le commerce électronique en Tunisie ?",

    # Questions pour des résumés et synthèses
    "Pouvez-vous résumer les dernières réformes législatives publiées dans le JORT ?",
    "Quels sont les points clés du dernier décret gouvernemental concernant la fiscalité ?",
    "Quelles sont les principales évolutions du droit du travail en Tunisie cette année ?",
    "Faites-moi une synthèse des dernières modifications du Code des investissements."
]

In [16]:
# Fonction de réponse du chatbot
def chatbot(question):
    if not question.strip():
        return "Veuillez entrer une question."

    response = qa_chain.invoke({"query": question})
    answer = response['result']
    sources = "\n\n".join([doc.page_content for doc in response['source_documents']])

    return f"**Réponse :** {answer}\n\n**Sources :**\n{sources}"

In [17]:
# Interface Gradio
with gr.Blocks() as interface:
    gr.Markdown("## Chatbot IA - LangChain & Groq\nPosez votre question ou choisissez-en une prédéfinie.")

    with gr.Row():
        question_input = gr.Textbox(placeholder="Posez votre question ici...")
        submit_btn = gr.Button("Envoyer")

    output = gr.Markdown()

    # Boutons pour les questions prédéfinies
    gr.Markdown("### Questions Prédéfinies")
    for q in sample_questions:
        gr.Button(q).click(fn=chatbot, inputs=gr.Textbox(value=q, visible=False), outputs=output)

    # Lien entre l'entrée manuelle et le chatbot
    submit_btn.click(chatbot, inputs=question_input, outputs=output)

# Lancement de l'interface
interface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://78a28c6462b0185d39.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
